#Missing value imputation using Scikit_Learn for categorical and numerical variables using different
#Strategies SimpleImputer,ColumnTransform,Pipeline


In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.shape,test.shape

((1460, 81), (1459, 80))

In [3]:
x_train=train.drop(columns="SalePrice",axis=1 )
y_train=train["SalePrice"]
x_test=test.copy()
x_train.shape,y_train.shape,x_test.shape

((1460, 80), (1460,), (1459, 80))

In [4]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [13]:
isnull_sum=x_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [6]:
x_train.isnull().sum().sum()

6965

#Numerical Variable missing value imputation

In [9]:
num_vars=x_train.select_dtypes(include=["int64","float64"]).columns
num_vars

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [15]:
num_vars_miss=[var for var in num_vars if isnull_sum[var]>0]
num_vars_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

#for Categorical variable

In [16]:
cat_vars=x_train.select_dtypes(include="object").columns
cat_vars

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [17]:
cat_vars_miss=[var for var in cat_vars if isnull_sum[var]>0]
cat_vars_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

##suppose now ur strateragy is to impute mean value in LotFrontage and and median in MasVnrArea', 'GarageYrBlt,

#In categorical columns u want to impute mode for some columns and constant for some columns here we were not able to use simpleimputer thus we will first create pipeline ..



In [29]:
#for numerical

num_vars_mean=["LotFrontage"]
num_vars_median=["MasVnrArea","GarageYrBlt"]

#for categorical fill mode

cat_vars_mode=['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType'
 ]

 #for categorical fill constant

cat_vars_const=['GarageFinish', 
'GarageQual',
'GarageCond',
'PoolQC',
'Fence',
'MiscFeature']



#Now we will create a Pipeline

In [40]:
num_vars_mean_imputer=Pipeline(steps=[("Mean", SimpleImputer(strategy="mean"))])
num_vars_median_imputer=Pipeline(steps=[("Median", SimpleImputer(strategy="median"))])
cat_vars_mode_imputer=Pipeline(steps=[("Mode",SimpleImputer(strategy="most_frequent"))] )
cat_vars_constant_imputer=Pipeline(steps=[("Constant",SimpleImputer(strategy="constant", fill_value="missing"))])
#List of (name, transform) tuples (implementing fit/transform) that are chained, 
#in the order in which they are chained, with the last object an estimator.
#here we wil define stratergy..

#in steps we have two values 1 is steps and other is transform(stratergy)
#steps in list and name and transform in tuble
#now we have only created pipline applying different stratergies

In [41]:
#numerical value first name then which stratergy want to to apply on which column
preprocessor = ColumnTransformer(transformers=[("mean_imputer", num_vars_mean_imputer, num_vars_mean), 
                                ("median_imputer", num_vars_median_imputer, num_vars_median),
                               ("mode_imputer", cat_vars_mode_imputer, cat_vars_mode),
                                ("missing_imputer", cat_vars_constant_imputer,cat_vars_const)])

#transformer(name,estimator[passthrough:drop],columns ):

#1.name(you can give any name)
#2.estimator(which stratergy )
#3.column name

In [42]:
#fit this on train data only now we can see on which column which stratergy has applied 
preprocessor.fit(x_train) 

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('Mean', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('Median',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('Mode',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical', 'FireplaceQu',
                                  'GarageType']),
                                ('missing_imputer',
                              

In [43]:
#transform values
preprocessor.transform 

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('Mean', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('Median',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('Mode',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical', 'FireplaceQu',
                                  'GarageType']),
                                ('mis

# Now will find out mean, median and mode value calculated for columns using statistics_

In [44]:
preprocessor.named_transformers_["mean_imputer"].named_steps["Mean"].statistics_
#here we got mean value for column "LotFrontage"

array([70.04995837])

In [45]:
preprocessor.named_transformers_["median_imputer"].named_steps["Median"].statistics_
#here we got median value for num_var_median=['MasVnrArea', 'GarageYrBlt'] 



array([   0., 1980.])

In [46]:
preprocessor.named_transformers_["mode_imputer"].named_steps["Mode"].statistics_
#here we got mode value for 
#"""['Alley',
#'MasVnrType',
#'BsmtQual',
#'BsmtCond',
# 'BsmtExposure',
# 'BsmtFinType1',
# 'BsmtFinType2',
# 'Electrical',
# 'FireplaceQu',
# 'GarageType',]"""

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd',
       'Attchd'], dtype=object)

# Now will get clean data   after tranform

In [47]:
x_train_clean = preprocessor.transform(x_train)
x_test_clean = preprocessor.transform(x_test)
#all statistics transforming it to main data set

In [49]:
x_test_clean

array([[80.0, 0.0, 1961.0, ..., 'missing', 'MnPrv', 'missing'],
       [81.0, 108.0, 1958.0, ..., 'missing', 'missing', 'Gar2'],
       [74.0, 0.0, 1997.0, ..., 'missing', 'MnPrv', 'missing'],
       ...,
       [160.0, 0.0, 1960.0, ..., 'missing', 'missing', 'missing'],
       [62.0, 0.0, 1980.0, ..., 'missing', 'MnPrv', 'Shed'],
       [74.0, 94.0, 1993.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [50]:
x_train_clean #here we got 2d array

array([[65.0, 196.0, 2003.0, ..., 'missing', 'missing', 'missing'],
       [80.0, 0.0, 1976.0, ..., 'missing', 'missing', 'missing'],
       [68.0, 162.0, 2001.0, ..., 'missing', 'missing', 'missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'missing', 'missing', 'missing'],
       [75.0, 0.0, 1965.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

# Now our task is to convert this 2d array into dataframe

In [51]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('Mean', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('Median', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('Mode', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu',
   'GarageType']),
 ('missing_imputer', Pipeline(steps=[('Constant',
                   SimpleImputer(fill_value='missing', strategy='constant'))]), ['GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,

# remainder:
 'drop',
 [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   61,
   62,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   75,
   76,
   77,
   78,
   79])]
#in this we can see here  remainder = in remainder we have bydefault paramter drop=it means in our main x_train dataframe value which are # no missing values variables index
#not missing values, those columns got drop and we got index of those columns... 

# But we dont want to drop these columns 
#so have to give "passthrough":"drop" value to the remainder ,now we have data in 2d in which we had missing value present

# Now will create dataframe of 2d array

In [52]:
x_train_clean_miss_var = pd.DataFrame(x_train_clean, columns=num_vars_mean+num_vars_median+cat_vars_mode+cat_vars_const)
#IT should be in squence
#we have cleaned missing variables

In [53]:
x_train_clean_miss_var.head()
x_train_clean_miss_var.shape

(1460, 19)

In [54]:
x_train_clean_miss_var.isnull().sum().sum()

0

In [55]:
train["Alley"].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [57]:
x_train_clean_miss_var["Alley"].value_counts()
#now we can see difference in values 

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [58]:
train["MiscFeature"].value_counts() #here we see our main data count 

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [60]:
x_train_clean_miss_var["MiscFeature"].value_counts() #common

missing    1406
Shed         49
Gar2          2
Othr          2
TenC          1
Name: MiscFeature, dtype: int64

#END

# Create Clean x_train DataFrame with call variables

In [61]:
remainder_vars_index = [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   61,
   62,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   75,
   76,
   77,
   78,
   79]

In [62]:
len(remainder_vars_index)

61

In [63]:
remainder_vars = [isnull_sum.keys()[var_index] for var_index in remainder_vars_index]
remainder_vars

['Id',
 'MSSubClass',
 'MSZoning',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

In [64]:
len(remainder_vars)

61

In [65]:
for var_index in remainder_vars_index:
     print(isnull_sum.keys()[var_index])

Id
MSSubClass
MSZoning
LotArea
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
ExterQual
ExterCond
Foundation
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
Heating
HeatingQC
CentralAir
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
GarageCars
GarageArea
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold
SaleType
SaleCondition


In [67]:
train.isnull().sum().sum()

6965

In [68]:
x_train=pd.concat([x_train_clean_miss_var,train[remainder_vars]],axis=1)

In [69]:
x_train.shape

(1460, 80)

In [70]:
x_train.isnull().sum().sum()

0

# Create test DataFrame with missing value imputed variables


In [72]:
x_test_clean_miss_var = pd.DataFrame(x_test_clean, columns=num_vars_mean+num_vars_median+cat_vars_mode+cat_vars_const)
x_test_clean_miss_var.shape

(1459, 19)

In [75]:
x_test_clean_miss_var.isnull().sum().sum()

0

In [77]:
# concatinate x_test_clean_miss_var df and remainder_vars
x_test =  pd.concat([x_test_clean_miss_var,test[remainder_vars]], axis=1)
x_test.shape

(1459, 80)

In [78]:
x_test.isnull().sum().sum()

22

In [79]:
# 22 <= What is this, in X_test df still missing values as available but why 
#because we fill missing values in those columns which have missing value present in only X_train df
# Basicaly we get df then  find missing values variables then split df into X_train, X_test, y_train, y_test
# after that we fill missing value

# so if you have train and test df seperatly then first thing you should concatinate then find the missing 
# values variables it's is great strategy and carry on
# so you can try yourself

In [81]:
isnull_sum_test = x_test.isnull().sum()
isnull_sum_test

LotFrontage      0
MasVnrArea       0
GarageYrBlt      0
Alley            0
MasVnrType       0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64

In [82]:
# finding the numerical variable which have mising value
num_vars_test = x_test.select_dtypes(include=["int64", "float64"]).columns
num_vars_miss_test = [var for var in num_vars_test if isnull_sum_test[var]>0]
num_vars_miss_test

['BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageCars',
 'GarageArea']

In [83]:
# finding the categorical variable which have mising value
cat_vars_test = x_test.select_dtypes(include=["O"]).columns
cat_vars_miss_test = [var for var in cat_vars_test if isnull_sum_test[var]>0]
cat_vars_miss_test

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'KitchenQual',
 'Functional',
 'SaleType']

#END